In [1]:
import os
import sys
sys.path.append('/user_data/isaacl/cosyne-submission-runs/modeling/')
sys.path.append('/user_data/isaacl/cosyne-submission-runs/')
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torch import nn
from torch import Tensor
from PIL import Image
from torchvision.transforms import Compose, Resize, ToTensor
from einops import rearrange, reduce, repeat
from einops.layers.torch import Rearrange, Reduce
from torchsummary import summary
from torch.utils.data import Dataset,DataLoader
from utils import *

from modeling.model_1 import *
from modeling.model_2 import *
from modeling.model_3 import *
from modeling.model_4 import *
from modeling.model_5 import *
from modeling.model_6 import *
from modeling.model_7 import *
from modeling.model_8 import *
from modeling.model_9 import *
from modeling.model_10 import *
from modeling.model_11 import *
from modeling.model_12 import *
from modeling.model_13 import *
from modeling.model_14 import *
from modeling.model_15 import *
from modeling.model_16 import *
from modeling.model_17 import *

In [ ]:
site = "m1s1"

vimg = np.load(f'../../all_sites_data_prepared/pics_data/val_img_{site}.npy')
vresp = np.load(f'../../all_sites_data_prepared/New_response_data/valRsp_{site}.npy')
vimg = np.reshape(vimg,(1000,1,50,50))

dic = dict()
# get_net = [net_one_neuron_m1,net_one_neuron_m2,net_one_neuron_m3,net_one_neuron_m4,net_one_neuron_m5,
#             net_one_neuron_m6,net_one_neuron_m7,net_one_neuron_m8,net_one_neuron_m9,net_one_neuron_m10,
#              net_one_neuron_m11,net_one_neuron_m13,net_one_neuron_m13,net_one_neuron_m14,net_one_neuron_m15,
#               net_one_neuron_m16,net_one_neuron_m17] #index 0 is model num 1
get_net = [model_2_one_neuron] #index 0 is model num 
# to_plot = [] #(neuron num, pc, plot)

In [ ]:
UNDERWORK STILL

for model_num in range(2,3):
    dic[model_num] = []
    rootdir = f'../results/{site}/model_{model_num}'

    for subdir, dirs, files in os.walk(rootdir):
        if 'model' not in files[0] or subdir[-3:] == 'avg':
            continue

        neuron_number = int(subdir.split("_")[-1])

        files.sort(key = lambda x: int(x.split(".")[0].split("_")[-1]),reverse=True)

        net = model_2_one_neuron()
        net.load_state_dict(torch.load(subdir + "/" + files[0], map_location=torch.device('cpu')))

        net_resp = torch.from_numpy(np.float32(vimg))
        with torch.no_grad():
            net.eval()
            net_resp = net(net_resp).detach().numpy()
        net_resp = np.transpose(net_resp)[0]

        real_resp = vresp[:,neuron_number]
        real_resp = np.transpose(real_resp)

        comb_resp_ord = []
        for r,n in zip(real_resp,net_resp):
            comb_resp_ord.append((r,n))
        comb_resp_ord.sort(key = lambda x:x[0],reverse=True)

        dic[model_num].append((neuron_number,pc_one_neuron(net,neuron_number,vimg,vresp), comb_resp_ord))
    dic[model_num].sort(key= lambda x: x[0])
    np.save(f'backup_{model_num}_{site}_pc_cro',dic)

np.save(f'{site}_pc_cro',dic)